In [ ]:
# https://www.kaggle.com/kinguistics/classifying-news-headlines-with-scikit-learn/notebook
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import pickle
#import sklearn.externals
#import joblib

#from joblib import dump, load
#from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
#from wordcloud import ImageColorGenerator, STOPWORDS, WordCloud

In [ ]:
# read to dataframe
news = pd.read_json("news_v02.json")

print("title count:", len(news))
print(news.groupby(["topic"]).count())
news.sample(10)

In [ ]:
def normalize_text(s):
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s

news['text'] = [normalize_text(s) for s in news['title']]
print("done normalizing text")
news.sample(10)

In [ ]:
# pull the data into vectors
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(news['text'])
# persist vectorizer
with open("pkl-vectorizer.pkl", 'wb') as file :
    pickle.dump(vectorizer, file)
#joblib.dump(vectorizer, "vectorizer.joblib")
#
#vocab = vectorizer.vocabulary_
#print(vocab)

encoder = LabelEncoder()
y = encoder.fit_transform(news['topic'])

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
#print(x_test.shape)

print("DONE vectorizing text")

In [ ]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

# save model
with open("model.pkl", 'wb') as model_file :
    pickle.dump(nb, model_file)
#joblib.dump(nb, "mnb-news_gCloud.joblib")

# load model
'''nb_load = joblib.load("mnb-news_gCloud.joblib")

print(nb_load.score(x_test, y_test))
'''
print("\nDONE!")

In [ ]:
# wordcloud
text = [key for key in vocab]
text = " ".join(text)
stop = set(STOPWORDS)
stop.add("will")
wordcloud = WordCloud(stopwords = stop, background_color = "white").generate(text)

plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()